In [15]:
import pandas as pd
from sklearn.model_selection import KFold
import json

In [17]:
main_path = r'../..'

import sys
sys.path.append(main_path)
from BayDS import *

Using TensorFlow backend.
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [3]:
Xscore = pd.read_csv('oof_best_submit.csv').set_index('TransactionID')

In [5]:
X_score_agg =  pd.read_csv('X_score_agg.csv').set_index('TransactionID')

In [12]:
y = pd.read_pickle(f'f:/my/Prog/kaggle/Baydin/Data/y.pkl')


In [10]:
all_features = list(X_score_agg.columns)

In [39]:
n_fold = 5
folds = KFold(n_splits=n_fold)
params = {
#           'num_leaves': 491,
#           'min_child_weight': 0.03454472573214212,
#           'feature_fraction': 0.3797454081646243,
#           'bagging_fraction': 0.4181193142567742,
#           'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
#           'reg_alpha': 0.3899927210061127,
#           'reg_lambda': 0.6485237330340494,
          'random_state': 47, 
         }

train_options = {
    "model_type":'lgb',
    "params": params,
    "eval_metric":'auc',
    'early_stopping_rounds': 200,
    'n_estimators': 1000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': 1
}

In [40]:
#Compute original score
splitter = folds.split(X, y)
for fold_n, (train_index, valid_index) in enumerate(splitter):
    gc.collect()
    if fold_n < folds.n_splits - train_options['n_splits']:
        continue
    print(fold_n)
    auc_baseline = fast_auc(y.iloc[valid_index], X_score_agg['score'].iloc[valid_index])
print("Baseline auc is", auc_baseline)

4
Baseline auc is 0.9270951865483884


In [34]:
y

TransactionID
2987000    0
2987001    0
2987002    0
2987003    0
2987004    0
          ..
3577535    0
3577536    0
3577537    0
3577538    0
3577539    0
Name: isFraud, Length: 590540, dtype: uint8

In [45]:
with open('../../Data/uplift_from_aggregates.csv','a') as logger:
    logger.write('feature_name, uplift\n')

In [48]:
for fname in all_features:
    if fname == 'score':
        continue
    print(fname) 
    X = X_score_agg[['score',fname]]
    result_dict_lgb = train_model_classification(X=X, X_test=None, y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=False,
                                             verbose=-1, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             splits = train_options['n_splits'],
                                             n_jobs=-1, groups=None)
    uplift = result_dict_lgb['scores'][0] - auc_baseline
    print(f'{fname} - {uplift}') 
    with open('../../Data/uplift_from_aggregates.csv','a') as logger:
        logger.write(f'{fname}, {uplift}\n')
    

TransactionAmt_by_card_id_ws1d_to_mean
Fold 5 started at Wed Sep 11 21:14:45 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[114]	training's auc: 0.936864	training's auc: 0.936909	valid_1's auc: 0.927138	valid_1's auc: 0.92719
CV mean score: 0.9271, std: 0.0000.
TransactionAmt_by_card_id_ws1d_to_mean - 4.2825412114866346e-05
TransactionAmt_by_card_id_ws1d_to_std
Fold 5 started at Wed Sep 11 21:15:22 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[157]	training's auc: 0.938618	training's auc: 0.938618	valid_1's auc: 0.92671	valid_1's auc: 0.926722
CV mean score: 0.9267, std: 0.0000.
TransactionAmt_by_card_id_ws1d_to_std - -0.0003847393511883235
TransactionAmt_by_card_id_ws2d_to_mean
Fold 5 started at Wed Sep 11 21:16:04 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[102]	training's auc: 0.936522	training's auc: 0.936556	

Fold 5 started at Wed Sep 11 21:28:09 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[32]	training's auc: 0.935524	training's auc: 0.935677	valid_1's auc: 0.926704	valid_1's auc: 0.92698
CV mean score: 0.9267, std: 0.0000.
C1_by_card_id_ws7d_to_mean - -0.0003913222331373145
C1_by_card_id_ws7d_to_std
Fold 5 started at Wed Sep 11 21:28:49 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[28]	training's auc: 0.935667	training's auc: 0.935808	valid_1's auc: 0.92618	valid_1's auc: 0.926299
CV mean score: 0.9262, std: 0.0000.
C1_by_card_id_ws7d_to_std - -0.0009149332075032435
C1_by_card_id_ws30d_to_mean
Fold 5 started at Wed Sep 11 21:29:28 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[34]	training's auc: 0.935697	training's auc: 0.935833	valid_1's auc: 0.927044	valid_1's auc: 0.927343
CV mean score: 0.9270, std: 0.0000.
C1_by

Fold 5 started at Wed Sep 11 21:46:08 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[30]	training's auc: 0.935345	training's auc: 0.935494	valid_1's auc: 0.926332	valid_1's auc: 0.926677
CV mean score: 0.9263, std: 0.0000.
C2_by_card_id_ws10_to_std - -0.0007627674423629216
C2_by_card_id_ws100_to_mean
Fold 5 started at Wed Sep 11 21:46:54 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[32]	training's auc: 0.935556	training's auc: 0.935858	valid_1's auc: 0.92676	valid_1's auc: 0.927102
CV mean score: 0.9268, std: 0.0000.
C2_by_card_id_ws100_to_mean - -0.00033473987074195133
C2_by_card_id_ws100_to_std
Fold 5 started at Wed Sep 11 21:47:40 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[14]	training's auc: 0.934702	training's auc: 0.934795	valid_1's auc: 0.926438	valid_1's auc: 0.92623
CV mean score: 0.9264, std: 0.0000.
C2

Fold 5 started at Wed Sep 11 22:03:11 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[86]	training's auc: 0.937113	training's auc: 0.937168	valid_1's auc: 0.926233	valid_1's auc: 0.926507
CV mean score: 0.9262, std: 0.0000.
C4_by_card_id_ws3d_to_mean - -0.0008622421029236271
C4_by_card_id_ws3d_to_std
Fold 5 started at Wed Sep 11 22:04:00 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[59]	training's auc: 0.936223	training's auc: 0.936324	valid_1's auc: 0.926238	valid_1's auc: 0.926423
CV mean score: 0.9262, std: 0.0000.
C4_by_card_id_ws3d_to_std - -0.0008572234915131816
C4_by_card_id_ws7d_to_mean
Fold 5 started at Wed Sep 11 22:04:44 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[87]	training's auc: 0.937568	training's auc: 0.937629	valid_1's auc: 0.926149	valid_1's auc: 0.926284
CV mean score: 0.9261, std: 0.0000.
C4_b

Fold 5 started at Wed Sep 11 22:23:56 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[93]	training's auc: 0.936241	training's auc: 0.936388	valid_1's auc: 0.926928	valid_1's auc: 0.927047
CV mean score: 0.9269, std: 0.0000.
C6_by_card_id_ws5_to_std - -0.00016764341301345187
C6_by_card_id_ws10_to_mean
Fold 5 started at Wed Sep 11 22:24:53 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[36]	training's auc: 0.935232	training's auc: 0.935365	valid_1's auc: 0.926955	valid_1's auc: 0.927282
CV mean score: 0.9270, std: 0.0000.
C6_by_card_id_ws10_to_mean - -0.00014021042299738884
C6_by_card_id_ws10_to_std
Fold 5 started at Wed Sep 11 22:25:40 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[34]	training's auc: 0.935264	training's auc: 0.935409	valid_1's auc: 0.926636	valid_1's auc: 0.926925
CV mean score: 0.9266, std: 0.0000.
C6_

...

Fold 5 started at Thu Sep 12 05:40:37 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[31]	training's auc: 0.935271	training's auc: 0.93547	valid_1's auc: 0.926291	valid_1's auc: 0.92661
CV mean score: 0.9263, std: 0.0000.
D8_by_card_id_ws100_to_std - -0.000803783694689475
D9_by_card_id_ws1d_to_mean
Fold 5 started at Thu Sep 12 05:41:58 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[120]	training's auc: 0.936594	training's auc: 0.936649	valid_1's auc: 0.926558	valid_1's auc: 0.926604
CV mean score: 0.9266, std: 0.0000.
D9_by_card_id_ws1d_to_mean - -0.0005375383761950792
D9_by_card_id_ws1d_to_std
Fold 5 started at Thu Sep 12 05:43:50 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[120]	training's auc: 0.936651	training's auc: 0.936671	valid_1's auc: 0.926734	valid_1's auc: 0.926949
CV mean score: 0.9267, std: 0.0000.
D9_b

Fold 5 started at Thu Sep 12 06:12:46 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[13]	training's auc: 0.93577	training's auc: 0.936016	valid_1's auc: 0.926794	valid_1's auc: 0.926674
CV mean score: 0.9268, std: 0.0000.
D10_by_card_id_ws7d_to_mean - -0.00030081785500923885
D10_by_card_id_ws7d_to_std
Fold 5 started at Thu Sep 12 06:13:51 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[19]	training's auc: 0.93581	training's auc: 0.935982	valid_1's auc: 0.926012	valid_1's auc: 0.925801
CV mean score: 0.9260, std: 0.0000.
D10_by_card_id_ws7d_to_std - -0.001082959597097144
D10_by_card_id_ws30d_to_mean
Fold 5 started at Thu Sep 12 06:14:58 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[13]	training's auc: 0.935869	training's auc: 0.935963	valid_1's auc: 0.92673	valid_1's auc: 0.926621
CV mean score: 0.9267, std: 0.0000.
D1

Fold 5 started at Thu Sep 12 06:43:12 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[56]	training's auc: 0.936391	training's auc: 0.936399	valid_1's auc: 0.927571	valid_1's auc: 0.927798
CV mean score: 0.9276, std: 0.0000.
D11_by_card_id_ws10_to_std - 0.00047577234487750086
D11_by_card_id_ws100_to_mean
Fold 5 started at Thu Sep 12 06:44:34 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[57]	training's auc: 0.936364	training's auc: 0.936479	valid_1's auc: 0.927325	valid_1's auc: 0.927584
CV mean score: 0.9273, std: 0.0000.
D11_by_card_id_ws100_to_mean - 0.00022967590972633456
D11_by_card_id_ws100_to_std
Fold 5 started at Thu Sep 12 06:45:56 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[74]	training's auc: 0.936947	training's auc: 0.936986	valid_1's auc: 0.927095	valid_1's auc: 0.9274
CV mean score: 0.9271, std: 0.0000.

Fold 5 started at Thu Sep 12 07:17:52 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[79]	training's auc: 0.93557	training's auc: 0.935709	valid_1's auc: 0.926442	valid_1's auc: 0.926704
CV mean score: 0.9264, std: 0.0000.
D13_by_card_id_ws3d_to_mean - -0.0006529189711141914
D13_by_card_id_ws3d_to_std
Fold 5 started at Thu Sep 12 07:19:30 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[159]	training's auc: 0.936551	training's auc: 0.936545	valid_1's auc: 0.926562	valid_1's auc: 0.92669
CV mean score: 0.9266, std: 0.0000.
D13_by_card_id_ws3d_to_std - -0.0005328736135686851
D13_by_card_id_ws7d_to_mean
Fold 5 started at Thu Sep 12 07:21:29 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[143]	training's auc: 0.936308	training's auc: 0.936312	valid_1's auc: 0.926932	valid_1's auc: 0.92688
CV mean score: 0.9269, std: 0.0000.
D

Fold 5 started at Thu Sep 12 07:56:12 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[108]	training's auc: 0.935779	training's auc: 0.935866	valid_1's auc: 0.926642	valid_1's auc: 0.92682
CV mean score: 0.9266, std: 0.0000.
D14_by_card_id_ws5_to_std - -0.00045299980225899095
D14_by_card_id_ws10_to_mean
Fold 5 started at Thu Sep 12 07:58:28 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[21]	training's auc: 0.934901	training's auc: 0.935056	valid_1's auc: 0.926302	valid_1's auc: 0.92661
CV mean score: 0.9263, std: 0.0000.
D14_by_card_id_ws10_to_mean - -0.0007936019116946413
D14_by_card_id_ws10_to_std
Fold 5 started at Thu Sep 12 08:01:57 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[137]	training's auc: 0.936145	training's auc: 0.936151	valid_1's auc: 0.926755	valid_1's auc: 0.926865
CV mean score: 0.9268, std: 0.0000.


In [43]:
result_dict_lgb

{'oof': array([[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.00503485],
        [0.07737692],
        [0.0080698 ]]), 'scores': [0.9271380119605033]}